### **Question 1**

In [57]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as split_data
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [58]:
dataset = pd.read_csv("boston.csv", header=0)
dataset.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [59]:
x = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

In [60]:
train_x, test_x, train_y, test_y = split_data(x, y, train_size=0.75, shuffle=True)

In [61]:
def ReLU(vector):
    return np.maximum(0, vector)

def initializeParams(sizes):
    params = {}
    for i in range(1, len(sizes)):
        params['W' + str(i)] = np.random.rand(sizes[i], sizes[i-1])*0.01
        params['B' + str(i)] = np.random.rand(sizes[i], 1)*0.01
    return params

def forward_propagation(x, params):
    layers = len(params)//2
    weights = {}
    for i in range(1, layers + 1):
        weights['Z'+str(i)] = np.dot(params['W'+str(i)], x) + params['B'+str(i)] if i == 1 else np.dot(params['W'+str(i)], weights['A'+str(i-1)]) + params['B'+str(i)]
        weights['A'+str(i)] = weights['Z'+str(i)] if i == layers else ReLU(weights['Z'+str(i)])
    return weights

def compute_cost(weights, y):
    layers = len(weights)//2
    predicted_y = weights['A' + str(layers)]
    cost = (1/(2*len(y)))*np.sum(np.square(predicted_y - y))
    return cost

def backward_propagation(x, y, weights, params):
    layers = len(params)//2
    m = len(y)
    gradients = {}
    for i in range(layers, 0, -1):
        dA = np.dot(params['W'+str(i+1)].T, dZ) if (i!=layers) else (1/m)*(weights['A'+str(i)]-y)
        dZ = np.multiply(dA, np.where(weights['A'+str(i)]>=0, 1, 0)) if (i!=layers) else dA
        gradients['W'+str(i)] = (1/m)*np.dot(dZ, weights['A'+str(i-1)].T) if (i!=1) else (1/m)*np.dot(dZ, x.T)
        gradients['B'+str(i)] = (1/m)*np.sum(dZ, axis=1, keepdims=True)
    return gradients

def updating_params(params, gradients, alpha):
    layers = len(params)//2
    updated_params = {}
    for i in range(1, layers+1):
        updated_params['W'+str(i)] = params['W'+str(i)] - alpha*(gradients['W'+str(i)])
        updated_params['B'+str(i)] = params['B'+str(i)] - alpha*(gradients['B'+str(i)])
    return updated_params

def train_model(x, y, layerSize, epochs, alpha):
    params = initializeParams(layerSize)
    for i in range(epochs):
        forward_predict = forward_propagation(x.T, params)
#         cost = compute_cost(forward_predict, y.T)
        gradients = backward_propagation(x.T, y.T, forward_predict, params)
        params = updating_params(params, gradients, alpha)
    return params

def compute_accuracy(train_x, test_x, train_y, test_y, params, layerSizes):
    values_train = forward_propagation(train_x.T, params)
    values_test = forward_propagation(test_x.T, params)
    train_acc = np.sqrt(mean_squared_error(train_y, values_train['A'+str(len(layer_sizes)-1)].T))
    test_acc = np.sqrt(mean_squared_error(test_y, values_test['A'+str(len(layer_sizes)-1)].T))
    return train_acc, test_acc

def predict(x, params):
    values = forward_propagation(x.T, params)
    predictions = values['A'+str(len(values)//2)].T
    return predictions

In [62]:
epochs = 1000
alpha = 0.04
layerSize = [13, 5, 5, 1]

In [65]:
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)
print(x.shape, y.shape)

(379, 13) (379,) (127, 13) (127,)
(506, 13) (506,)


In [63]:
parameters = train_model(train_x, train_y, layerSize, epochs, alpha)

ValueError: Length of values (1) does not match length of index (379)